In [1]:
import pandas as pd
from core.MV2 import MV2
from core.JIG import JIG
from core.SG2 import SG2 
from core.AWL import AWL

strings = './datasets/pe_string.pkl'

strings_data = pd.read_pickle(strings)

In [2]:
import pickle

def main(payload_path, signature_path, stopword_path, virtual_vector_path, big_group_path,
         window_size, K, M, # MV2 파라미터
         thetaJ,  # JIG 파라미터
         vector_size, eps, minpts, ngram, hh1_size, hh2_size, hh3_size, ratio # SG2, AWL 파라미터
        ):
    
    with open(payload_path, 'rb') as f:
        payloads_data = pickle.load(f)

    print(len(payloads_data))
    payloads = [x for x in payloads_data]

    print(len(payloads))

    # 빅 그룹 식별
    minhashed_virtual_vectors = MV2(payloads=payloads, window_size=window_size, K=K, M=M)
    with open(virtual_vector_path, 'wb') as f:
        pickle.dump(minhashed_virtual_vectors, f)

    big_group_indices = JIG(vectors=minhashed_virtual_vectors, thetaJ=thetaJ)
    with open(big_group_path, 'wb') as f:
        pickle.dump(big_group_indices, f)
    
    big_group_payloads = []
    non_big_group_paylaods = []

    for idx, payload in enumerate(payloads):
        if idx in big_group_indices:
            big_group_payloads.append(payload)
        else:
            non_big_group_paylaods.append(payload)
    
    # 시그니처 생성
    cluster_signatures = SG2(payloads=big_group_payloads, window_size=window_size, vector_size=vector_size, 
                             eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
    
    with open(signature_path, 'wb') as f:
        pickle.dump(cluster_signatures, f)
    
    stopwords = AWL(payloads=non_big_group_paylaods, ngram=ngram, hh1_size=hh1_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)

    with open(stopword_path, 'wb') as f:
        pickle.dump(stopwords, f)


if __name__ == '__main__':

    file_path = '../PE/datasets/pe_string.pkl'
    virtual_vector_path = f'./res/PE_virtual_vector.pkl'
    big_group_indices_path = f'./res/PE_big_group.pkl'
    signature_path = f'./res/PE_signature.pkl'
    stopword_path = f'./res/PE_stopword.pkl'

    K = 64
    M = 2 ** 14
    thetaJ = 0.6
    window_size = 4
    vector_size = 512
    eps = 0.4
    minpts = 5
    ngram = 4
    hh1_size = 3000
    hh2_size = 3000
    hh3_size = 3000
    ratio = 0.1

    main(payload_path=file_path, signature_path=signature_path, stopword_path=stopword_path,
         virtual_vector_path=virtual_vector_path, big_group_path=big_group_indices_path,
         window_size=window_size, vector_size=vector_size, K=K, M=M, thetaJ=thetaJ,
         eps=eps, minpts=minpts, ngram=ngram, hh1_size=hh2_size, hh2_size=hh2_size, hh3_size=hh3_size, ratio=ratio)
    

29909
29909
make minhashed vector


100%|██████████| 29909/29909 [02:38<00:00, 189.08it/s]


checking big group


100%|██████████| 29909/29909 [02:24<00:00, 206.46it/s]


chunking


100%|██████████| 23402/23402 [00:31<00:00, 734.09it/s]


start DBSCAN
end DBSCAN
make signature


  0%|          | 0/3 [00:00<?, ?it/s]


TypeError: 'set' object is not subscriptable

In [26]:
import pickle
with open('./res/PE_signature.pkl', 'rb') as f:
    signature = pickle.load(f)
    
with open('./res/PE_stopword.pkl', 'rb') as f:
    stopword = pickle.load(f)
    
with open('./res/PE_big_group.pkl', 'rb') as f:
    big_group = pickle.load(f)

In [27]:
print(len(big_group))

22566


In [11]:
print('SIGNAURE')

for i in signature.values():
    for j in i:
        print(j, end=" ")
    print()
    
print('STOPWORD')
for i in stopword:
    print(i)

SIGNAURE
('$&-[-o', 832) 
('-H@{c!', 340) 
('d!`\\rr', 1008) 
('TRegistryU', 966) ('__vbaVarLateMemCallLd', 409) ('L8!H@w', 99) 
('open file', 607) 
('RtlDecompressBuffer', 510) 
('l(lBt]', 312) ('Inverflow', 156) ('symname(', 81) ('{ezxyxf+:$x', 12) 
('l][tSc', 2184) 
('module_main', 919) 
('FPlt68', 924) 
('v: 1.31', 742) ('CreateProcessA', 31) 
('StartMailWorm', 852) 
('TComponent', 1040) ('__vbaFreeStrList', 351) 
('           ', 605) 
('Wm>sdw', 701) ('Websit', 478) 
('       ', 635) 
('GetLocaleInfoEx', 465) ('.adata', 288) ('0v) `@', 45) 
('"?O=N(', 817) 
('     ', 330) 
('1B2R2_2h2n2x2~2', 344) 
('HbB<!T', 350) 
('_t$@SDI', 241) 
('Iac}JDTb2`A@', 244) 
('GetLongPathNameA', 227) ('FindNextFileA', 150) 
('EncodePointer', 432) 
('%U|mBk', 260) 
('4N:.a:', 169) 
('Init*K4', 269) 
('1 1$1(1,1014181<1@1D1H1L1P1T1X1\\1`1d1h1l1p1t1x1|1', 236) 
('BBBBRV', 110) 
("d,I'0ld6", 184) 
('@A_A^_^]', 156) 
('`-Q_Dv', 194) 
('\\ShowYJ', 166) 
('_cxx1', 89) 
('iiiiiiiiiii', 176) ('a2AQVCO', 14) 


In [12]:
filters = [i[0] for i in stopword if i[1] > 99]
for cluster, sig in signature.items():
    for token in sig:
        if token[1] not in filters:
            print(token)

('$&-[-o', 832)
('-H@{c!', 340)
('d!`\\rr', 1008)
('TRegistryU', 966)
('__vbaVarLateMemCallLd', 409)
('L8!H@w', 99)
('open file', 607)
('RtlDecompressBuffer', 510)
('l(lBt]', 312)
('Inverflow', 156)
('symname(', 81)
('{ezxyxf+:$x', 12)
('l][tSc', 2184)
('module_main', 919)
('FPlt68', 924)
('v: 1.31', 742)
('CreateProcessA', 31)
('StartMailWorm', 852)
('TComponent', 1040)
('__vbaFreeStrList', 351)
('           ', 605)
('Wm>sdw', 701)
('Websit', 478)
('       ', 635)
('GetLocaleInfoEx', 465)
('.adata', 288)
('0v) `@', 45)
('"?O=N(', 817)
('     ', 330)
('1B2R2_2h2n2x2~2', 344)
('HbB<!T', 350)
('_t$@SDI', 241)
('Iac}JDTb2`A@', 244)
('GetLongPathNameA', 227)
('FindNextFileA', 150)
('EncodePointer', 432)
('%U|mBk', 260)
('4N:.a:', 169)
('Init*K4', 269)
('1 1$1(1,1014181<1@1D1H1L1P1T1X1\\1`1d1h1l1p1t1x1|1', 236)
('BBBBRV', 110)
("d,I'0ld6", 184)
('@A_A^_^]', 156)
('`-Q_Dv', 194)
('\\ShowYJ', 166)
('_cxx1', 89)
('iiiiiiiiiii', 176)
('a2AQVCO', 14)
('Content-Type: text/plain', 66)
('*BTj1oR', 